<a href="https://colab.research.google.com/github/drdww/OPIM5641/blob/main/Module3/M3_1/BlendingModels_Coffee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Programming: Blending Models (Coffee)

**OPIM 5641: Business Decision Modeling - Dept. of Operations and Information Management - University of Connecticut**

-------------------------------

Related Readings:
* `Pyomo Cookbook`: https://github.com/jckantor/ND-Pyomo-Cookbook/blob/master/notebooks/02.01-Production-Models-with-Linear-Constraints.ipynb
* `Powell`: Chapter 9 (Linear Optimization)

## Background
When we describe outcomes in terms of proportions,and when we place a ﬂoor(or ceiling) on one or more of those proportions, we are using blending constraints of a special type. 

Whenever we encounter a constraint in the form of a lower limit or an upper limit on a proportion, we can take the following steps: 
1. Write the fraction that expresses the constrained proportion. 
2. Write the inequality implied by the lower bound or upper bound. 
3. Multiply both sides of the inequality by the denominator and collect terms. 
4. The result is a linear inequality, ready to incorporate in the model. 

In general, the blending model involves mixing materials with different individual properties and describing the properties of the blend with weighted averages. We might be familiar with the phenomenon of mixing if we have spent time in a chemistry lab mixing ﬂuids with different concentrations, but the concept extends beyond lab work.

## Setup Your Environment/Imports

In [ ]:
# import modules

%matplotlib inline
from pylab import *

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc"))

from pyomo.environ import *

     |████████████████████████████████| 9.0MB 2.8MB/s 
     |████████████████████████████████| 256kB 46.0MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 163kB 47.6MB/s 
Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 144579 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coi

# Example: Coffee 
*Section 9.4.2 (Powell)*

**Description:** The Diaz Coffee Company blends three types of coffee beans (Brazilian, Colombian and Peruvian) into ground coffee to be sold at retail. Suppose that each type of bean has a distinct aroma and strength, and that the company has a chief taster who can rate the features on a scale from 1 to 100. The featues of the beans are tabulated below.

Bean | Aroma Rating | Strength Rating | Cost/lb. (USD) | Pounds available
--- | --- | --- | --- | ---
Brazilian | 75 | 15 | 0.50 USD | 1,500,000
Colombian | 60 | 20 | 0.60 USD | 1,200,000
Peruvian | 85 | 18 | 0.70 USD | 2,000,000

The company would like to create a blend that has an aroma rating of at least 78 and a strength rating of at least 16. Its supplies of the various beans are limited, however. The available quantities are listed above. All beans are delivered under a previously arranged purchase agreement. Diaz wants to make four million pounds of the blend at the lowest possible cost.


**Define the Objective Function**

$Cost = 0.5B + 0.6C + 0.7P$

**Write the Constraints**

$Min(Z) = 0.5B + 0.6C + 0.7P$

We know that we need to have an aroma of at least 78 and a strength of at least 16. In this case, a weighted average is used. 

$Aroma = (B(75) + C(60) + P(85)) / (B + C + P)$

And to impose a constraint we write
$Aroma = (B(75) + C(60) + P(85)) / (B + C + P) >= 78$

But we know that we can't use a proportion as a constraint in a linear model. Let's rewrite it into something the model can use. First, multiply both sides by (B + C + P) which yields
$(B(75) + C(60) + P(85)) >= 78(B + C + P)$

Collect terms on the lefthand side and we end up with this:
$-3B - 18C + 7P >= 0$

And now we recognize this as the same type of constraint from the Furniture example. 

In a similar fashion, we can also make a linear constraint for the blend having a strength of at least 16, which yields:
$-1B + 4C + 2P >= 0$

To summarize, our constraints are...
* Supply constraints (can't use what you don't have)

$B <= 1,500,000$

$C <= 1,200,000$

$P <= 2,000,000$

* Demand constraint (you must at least satisfy demand!)

$B + C + P >= 4,000,000$

* Blending constraints (must meet a certain strength)

$-3B - 18C + 7P >= 0$

$-1B + 4C + 2P >= 0$


Great! Now that your problem is defined - go code it up and solve it.

In [ ]:
# declare the model
model = ConcreteModel()

# declare decision variables
model.b = Var(domain=NonNegativeReals) # brazil
model.c = Var(domain=NonNegativeReals) # colombia
model.p = Var(domain=NonNegativeReals) # peru

# declare objective
model.cost = Objective(
                      expr = 0.5*model.b + 0.6*model.c + 0.7*model.p, # values come from the table
                      sense = minimize) # remember, you are minimizing cost!!!

# declare constraints
model.Constraint1 = Constraint(expr = model.b + model.c + model.p >= 4000000) # demand
model.Constraint2 = Constraint(expr = -3*model.b + -18*model.c + 7*model.p >= 0) # aroma
model.Constraint3 = Constraint(expr = -1*model.b + 4*model.c + 2*model.p >= 0) # stength
model.Constraint4 = Constraint(expr = model.b <= 1500000) # b supply
model.Constraint5 = Constraint(expr = model.c <= 1200000) # c supply
model.Constraint6 = Constraint(expr = model.p <= 2000000) # p supply

In [ ]:
# show the model you've created
model.pprint()

3 Var Declarations
    b : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    c : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    p : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    cost : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 0.5*b + 0.6*c + 0.7*p

6 Constraint Declarations
    Constraint1 : Size=1, Index=None, Active=True
        Key  : Lower     : Body      : Upper : Active
        None : 4000000.0 : b + c + p :  +Inf :   True
    Constraint2 : Size=1, Index=None, Active=True
        Key  : Lower : Body              : Upper : Active
        None :   0.0 : -3*b - 18*

In [ ]:
# solve it
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 2448000.0
  Upper bound: 2448000.0
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 4
  Number of nonzeros: 3
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of cre

In [ ]:
# show the results
print("Cost = ", model.cost(), " USD")
print("Brazilian Beans = ", model.b(), " lb")
print("Colombian Beans = ", model.c(), " lb")
print("Peruvian Beans = ", model.p(), " lb")

Cost =  2448000.0  USD
Brazilian Beans =  1500000.0  lb
Colombian Beans =  520000.0  lb
Peruvian Beans =  1980000.0  lb


In [ ]:
# do we need to show binding constraints here? easier way to do this?
print("Demand Constraint:", model.Constraint1())
print("Aroma Constraint:", model.Constraint2()) # don't worry if these look funky! more below.
print("Stength Constraint", model.Constraint3()) # don't worry if these look funky! 
print("B Supply Constraint:", model.Constraint4())
print("C Supply Constraint:", model.Constraint5())
print("P Supply Constraint", model.Constraint6())

Demand Constraint: 4000000.0
Aroma Constraint: 0.0
Stength Constraint 4540000.0
B Supply Constraint: 1500000.0
C Supply Constraint: 520000.0
P Supply Constraint 1980000.0


In [ ]:
# show that we got the ratio we need!

# strength - need at least 78
strength = (75*model.b() + 60*model.c() + 85*model.p())  /(model.b() + model.c() + model.p())
print(strength)

# aroma - need at least 16
aroma = (15*model.b() + 20*model.c() + 18*model.p())  /(model.b() + model.c() + model.p())
print(aroma)

78.0
17.135


We can be satisfied that we have our minimum strength and aroma.

# On Your Own
Try to explore shadow prices and make interesting graphs and tables.